# Traffic Signs Recognition Project with Tensorflow Serving

The dataset can be downloaded from Kaggle __[here](https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign?select=Train.csv)__.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import glob
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
data = []
labels = []
classes = 43
cur_path = '/path/to/dataset/directory/'

#Retrieving the images and their labels 
for i in range(classes):
    path = cur_path + 'Train/' + str(i) + '/*'
    images = glob.glob(path)
    for im in images:
        try:
            image = cv2.imread(im)
            image = cv2.resize(image, (30,30))
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)



In [ ]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))

Save the Model So That Inferences Can be Done Just Via Image Upload

In [ ]:
MODEL_DIR = '/path/to/model/traffic_signs_model'
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

## Check the Saved Model

In [ ]:
!saved_model_cli show --dir {export_path} --all

In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()



## Install Tensorflow Serving to Serve the Model (For Linux or Colab)

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''


!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update


!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

## Serve the Model

In [ ]:
tensorflow_model_server --port=8500 --rest_api_port=8598 --model_name=traffic_signs_model --model_base_path="/path/to/model/traffic_signs_model"

In [ ]:
data = []
labels = []
cur_path = '/path/to/dataset/directory/'

path = cur_path + 'Test/*'
images = glob.glob(path)

#Retrieving the images and their labels
for im in images[:5]:
    try:
        image = cv2.imread(im)
        image = cv2.resize(image, (30,30))
        data.append(image)
    except:
        print("Error loading image")
    

#Converting lists into numpy arrays
test_imgs = np.array(data)
data = json.dumps({"signature_name": "serving_default", "inputs": test_imgs.tolist()})
headers = {"content-type": "application/json"}
json_response = requests.post('http://127.0.0.1:8598/v1/models/traffic_signs_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['outputs']
print(predictions)

In [ ]:
#dictionary to label all traffic signs class.
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

### Get Predictions

In [ ]:
for pred in predictions:    
    print(np.argmax(pred), classes[np.argmax(pred)])

### See Predictions along with the Image Display

In [ ]:
for pred, img in zip(predictions, test_imgs):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Class ' + str(np.argmax(pred)) + ' : ' + classes[np.argmax(pred)])
    plt.show() 

## Test the Model on the Complete Testing Dataset and Compare Accuracies

In [ ]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('/path/to/dataset/directory/Test.csv')

labels = y_test["ClassId"].values

data = []
cur_path = '/path/to/dataset/directory/'

path = cur_path + 'Test/*'
images = glob.glob(path)

#Retrieving the images and their labels
for im in images[:5]:
    try:
        image = cv2.imread(im)
        image = cv2.resize(image, (30,30))
        data.append(image)
    except:
        print("Error loading image")
    

#Converting lists into numpy arrays
test_imgs = np.array(data)
data = json.dumps({"signature_name": "serving_default", "inputs": test_imgs.tolist()})
headers = {"content-type": "application/json"}
json_response = requests.post('http://127.0.0.1:8598/v1/models/traffic_signs_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['outputs']
print(predictions)


predicted_classes = []
for pred in predictions:    
    predicted_classes.append(np.argmax(pred))

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, predicted_classes))